In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime as dt
import lazypredict

In [15]:
# Try reading with utf-8 encoding

df = pd.read_csv('Dataset.csv')


In [16]:
#Import data
#df = pd.read_csv('FinalDatasetPreMidterm.xlsx - Sheet1.csv')
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df

,Date,CPI,NaN,Date,Investment in Housing,NaN,Date,GDP,NaN,Date,...,1y Govy,NaN,Date,10y Govy,NaN,DATE,China Discount Rate,NaN,Date,Current Accounts
1,9/30/2023,-3.2,NaN,9/30/2023,-9.1,NaN,9/30/2023,5.2,NaN,11/1/2023,...,2.233,NaN,11/1/2023,2.686,NaN,1990-03-01,9,NaN,6/30/2023,64727.7
2,8/31/2023,-1.7,NaN,8/31/2023,-8.8,NaN,6/30/2023,5.5,NaN,10/31/2023,...,2.204,NaN,10/31/2023,2.689,NaN,1990-04-01,9,NaN,3/31/2023,81529.7
3,7/31/2023,-1.7,NaN,7/31/2023,-8.5,NaN,3/31/2023,4.5,NaN,9/29/2023,...,2.126,NaN,9/29/2023,2.673,NaN,1990-05-01,9,NaN,12/30/2022,103116.2
4,6/30/2023,2.3,NaN,6/30/2023,-7.9,NaN,12/31/2022,3,NaN,8/31/2023,...,1.867,NaN,8/31/2023,2.579,NaN,1990-06-01,9,NaN,9/30/2022,139636.4
5,5/31/2023,1,NaN,5/31/2023,-7.2,NaN,9/30/2022,3,NaN,7/31/2023,...,1.809,NaN,7/31/2023,2.659,NaN,1990-07-01,9,NaN,6/30/2022,73773.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-04-01,2.9,NaN,NaN,NaN
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-05-01,2.9,NaN,NaN,NaN
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-06-01,2.9,NaN,NaN,NaN
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-07-01,2.9,NaN,NaN,NaN


In [17]:
#seperate data
data = dict()
for i in range(0,len(df.columns),3):
  data[df.columns[i+1]] = df.iloc[:,i:i+2]

In [18]:
data.keys()

dict_keys(['CPI', 'Investment in Housing', 'GDP', 'XAUUSD Curncy - Gold', 'Big China EtF', 'Brent Crude', 'WTI', 'Prime Loan 1y', 'Prime Loan 5y', 'Mining Value-Add', 'Manufacturing Value-Add', '6m Govy', '1y Govy', '10y Govy', 'China Discount Rate', 'Current Accounts'])

In [19]:
for dataset in data:
  data[dataset] = data[dataset].dropna()
  data[dataset].iloc[:,0] = pd.to_datetime(data[dataset].iloc[:,0])

In [20]:
#make this data End of Month
#Flip data to descending
data['China Discount Rate']['DATE'] = data['China Discount Rate']['DATE'] - timedelta(days=1)
data['China Discount Rate'].sort_values('DATE', ascending=0, inplace = True)
data['China Discount Rate'].rename(columns={"DATE":"Date"}, inplace=True)

In [21]:
#account for potential data discrepancies in dates including weekends
#I do this by making data purely monthly
for dataset in data:
  data[dataset]['Date'] =  data[dataset]['Date'].dt.to_period('M')

In [22]:
final_df = data['CPI']
for dataset in data:
  if dataset == 'CPI':
    continue
  else:
    final_df = final_df.merge(data[dataset], on='Date', how='outer')


In [23]:
#drop prime loan 1, prime loan 5, 6 month govy as data is too short
final_df = final_df.drop(columns=['6m Govy','Prime Loan 1y', 'Prime Loan 5y'])

In [24]:
#cut data
final_df = final_df[0:203] #cut to 2013 june
final_df = final_df[final_df['Date'].dt.year > 2012]
final_df = final_df[0:len(final_df)-5]
final_df

,Date,CPI,Investment in Housing,GDP,XAUUSD Curncy - Gold,Big China EtF,Brent Crude,WTI,Mining Value-Add,Manufacturing Value-Add,1y Govy,10y Govy,China Discount Rate,Current Accounts
0,2023-09,-3.2,-9.1,5.2,1848.63,18.92,95.31,90.79,1.5,5,2.126,2.673,NaN,NaN
1,2023-08,-1.7,-8.8,NaN,1940.19,19.37,86.86,83.63,2.3,5.4,1.867,2.579,NaN,NaN
2,2023-07,-1.7,-8.5,NaN,1965.09,21.2,85.56,81.8,1.3,3.9,1.809,2.659,2.9,NaN
3,2023-06,2.3,-7.9,5.5,1919.35,19.29,74.9,70.64,1.5,4.8,1.862,2.636,2.9,64727.7
4,2023-05,1,-7.2,NaN,1962.73,18.45,72.66,68.09,-1.2,4.1,1.989,2.706,2.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2013-10,6.5,19.2,NaN,1323.06,20.7,108.84,96.38,4.3,11.4,3.889,4.203,3.25,NaN
120,2013-09,6.1,19.7,7.8,1329.03,20.25,108.37,102.33,4.9,11.1,3.432,4.007,3.25,32587.4
121,2013-08,4.7,19.3,NaN,1395.27,19.24,114.01,107.65,5.8,10.9,3.6,4.018,3.25,NaN
122,2013-07,5,20.5,NaN,1325.07,18.78,107.7,105.03,5.5,10.5,3.448,3.732,3.25,NaN


In [25]:
#reset index
final_df.set_index('Date', inplace=True)
final_df.dtypes

0
CPI                        object
Investment in Housing      object
GDP                        object
XAUUSD Curncy - Gold       object
Big China EtF              object
Brent Crude                object
WTI                        object
Mining Value-Add           object
Manufacturing Value-Add    object
1y Govy                    object
10y Govy                   object
China Discount Rate        object
Current Accounts           object
dtype: object

In [26]:
#change datatypes and interpolate
final_df = final_df.astype(float)
final_df.interpolate(method='linear', inplace=True)
final_df

,CPI,Investment in Housing,GDP,XAUUSD Curncy - Gold,Big China EtF,Brent Crude,WTI,Mining Value-Add,Manufacturing Value-Add,1y Govy,10y Govy,China Discount Rate,Current Accounts
Date,,,,,,,,,,,,,
2023-09,-3.2,-9.1,5.200000,1848.63,18.92,95.31,90.79,1.5,5.0,2.126,2.673,NaN,NaN
2023-08,-1.7,-8.8,5.300000,1940.19,19.37,86.86,83.63,2.3,5.4,1.867,2.579,NaN,NaN
2023-07,-1.7,-8.5,5.400000,1965.09,21.20,85.56,81.80,1.3,3.9,1.809,2.659,2.90,NaN
2023-06,2.3,-7.9,5.500000,1919.35,19.29,74.90,70.64,1.5,4.8,1.862,2.636,2.90,64727.700000
2023-05,1.0,-7.2,5.166667,1962.73,18.45,72.66,68.09,-1.2,4.1,1.989,2.706,2.90,70328.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-10,6.5,19.2,7.800000,1323.06,20.70,108.84,96.38,4.3,11.4,3.889,4.203,3.25,32380.433333
2013-09,6.1,19.7,7.800000,1329.03,20.25,108.37,102.33,4.9,11.1,3.432,4.007,3.25,32587.400000
2013-08,4.7,19.3,7.766667,1395.27,19.24,114.01,107.65,5.8,10.9,3.600,4.018,3.25,35948.666667


In [27]:
#drop remaining nan
final_df = final_df.dropna()

In [28]:
#data stats
print("data length", len(final_df))
print("start_date", final_df.index[0], "end_date", final_df.index[-1])
final_df.describe()

data length 121
start_date 2023-06 end_date 2013-06


,CPI,Investment in Housing,GDP,XAUUSD Curncy - Gold,Big China EtF,Brent Crude,WTI,Mining Value-Add,Manufacturing Value-Add,1y Govy,10y Govy,China Discount Rate,Current Accounts
count,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000
mean,3.604959,7.350000,6.245455,1455.648347,23.957769,70.084959,64.912562,2.838017,6.557025,2.640760,3.286405,2.980992,55968.162810
std,5.109352,8.434959,3.483926,277.317653,4.864729,23.008312,21.444829,3.167606,2.789506,0.611575,0.491804,0.148219,36726.026943
min,-5.200000,-16.300000,-6.900000,1061.100000,15.300000,22.740000,18.840000,-3.800000,-4.600000,1.113000,2.509000,2.900000,-52251.500000
25%,0.700000,3.700000,6.000000,1244.550000,20.260000,52.380000,48.520000,0.500000,5.300000,2.215000,2.854000,2.900000,30634.100000
50%,2.700000,8.100000,6.900000,1321.250000,23.100000,66.030000,60.140000,3.000000,6.700000,2.516000,3.179000,2.900000,61588.400000
75%,5.000000,10.900000,7.400000,1765.940000,26.680000,84.380000,78.870000,4.700000,7.833333,3.139000,3.616000,2.900000,77863.633333
max,21.900000,38.300000,18.700000,1990.000000,37.720000,122.840000,114.670000,12.200000,15.200000,4.084000,4.576000,3.250000,139636.400000


##BUILD MODELS

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

####Test-Train split

In [30]:
X = final_df.drop(columns='China Discount Rate')
y = final_df['China Discount Rate']
# Split the dataset into training and testing sets
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

###Model 1: OLS Regression (Linear)

In [31]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np
model = LinearRegression()
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error


# Use cross_val_predict to get predictions for each fold
y_pred = cross_val_predict(model, X, y, cv=10)

# Calculate R-squared manually
r2 = r2_score(y, y_pred)

# Calculate Negative Mean Absolute Error manually
nae = -mean_absolute_error(y, y_pred)

# Use cross_val_score to get the individual R-squared values
r2_scores = cross_val_score(model, X, y, cv=10, scoring='r2')

# Use cross_val_score to get the individual Negative Mean Absolute Error values
nae_scores = cross_val_score(model, X, y, cv=10, scoring='neg_mean_absolute_error')

# Print the results
print("Overall R^2:", r2)
print("Overall Mean Absolute Error:", -nae)

Overall R^2: 0.5489393618453134
Overall Mean Absolute Error: 0.06885682186608988


### Use Lazypredict to suggest model ideas

In [32]:
from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np

X = final_df.drop(columns='China Discount Rate')
y = final_df['China Discount Rate']

# #sequential
# offset = int(X.shape[0] * 0.8)
# X_train, y_train = X[:offset], y[:offset]
# X_test, y_test = X[offset:], y[offset:]


#random
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.8,random_state =123)

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

 79%|███████▊  | 33/42 [00:00<00:00, 40.44it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:01<00:00, 37.15it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 3.031250
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

###Model 2: Randomforest Model

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# Create a random forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Use cross_val_predict to get predictions for each fold
y_pred_rf = cross_val_predict(rf_model, X, y, cv=10)

# Calculate R-squared manually
r2_rf = r2_score(y, y_pred_rf)

# Calculate Negative Mean Absolute Error manually
nae_rf = -mean_absolute_error(y, y_pred_rf)

# Use cross_val_score to get the individual R-squared values
r2_scores_rf = cross_val_score(rf_model, X, y, cv=10, scoring='r2')

# Use cross_val_score to get the individual Negative Mean Absolute Error values
nae_scores_rf = cross_val_score(rf_model, X, y, cv=10, scoring='neg_mean_absolute_error')

# Print the results
print("Overall R^2 (Random Forest):", r2_rf)
print("Overall Negative Mean Absolute Error (Random Forest):", nae_rf)
#print("Individual R^2 scores (Random Forest):", r2_scores_rf)
#print("Individual Negative Mean Absolute Error scores (Random Forest):", nae_scores_rf)


Overall R^2 (Random Forest): 0.7030620967741936
Overall Negative Mean Absolute Error (Random Forest): -0.04544214876033138


####Hyper-parameterize the model

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
import numpy as np


# Create a Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameters and their potential values
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    # Add more hyperparameters as needed
}

# Define the scoring metrics (negative mean absolute error and R²)
nae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
r2_scorer = 'r2'  # Scikit-learn's default r2_score is used here

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(rf_model, param_grid, scoring={'NAE': nae_scorer, 'R2': r2_scorer}, refit='NAE', cv=10)
grid_search.fit(X, y)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Get the best model
best_rf_model = grid_search.best_estimator_

# Use cross_val_score to get the overall performance of the best model (NAE)
overall_nae_performance = cross_val_score(best_rf_model, X, y, cv=10, scoring=nae_scorer)

# Fit the best model to the entire dataset to get R²
best_rf_model.fit(X, y)
y_pred = best_rf_model.predict(X)
r2_best_model = r2_score(y, y_pred)

# Print the results
print("Best Hyperparameters:", best_params)
print("Overall Negative Mean Absolute Error (Best Model):", np.mean(overall_nae_performance))
print("R² for the Best Model:", r2_best_model)

Best Hyperparameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Overall Negative Mean Absolute Error (Best Model): -0.041847435897438154
R² for the Best Model: 0.9946284178187399


In [35]:
y_test_pred = best_rf_model.predict(X_test)
print("test r2:",r2_score(y_test, y_test_pred))
print("test mae:",mean_absolute_error(y_test, y_test_pred))

test r2: 0.9956801619433194
test mae: 0.002381443298972106


In [36]:
#feature importance rf

# Get feature importances
feature_importances = best_rf_model.feature_importances_

#get feature names
feature_names = [f'{X.columns[i]}' for i in range(len(X.columns))]

# Create a DataFrame to display the feature importances along with the feature names
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the top 5 most important features
top_features = importance_df.head(5)
print("Top 5 Most Important Features:")
print(top_features)


Top 5 Most Important Features:
                 Feature  Importance
2                    GDP        0.60
3   XAUUSD Curncy - Gold        0.18
10              10y Govy        0.14
0                    CPI        0.03
4          Big China EtF        0.03


###Model 3: GradientBoostingRegressor

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error


# Create a Gradient Boosting model
gb_model = GradientBoostingRegressor(random_state=69)

# Define the hyperparameters and their potential values
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    # Add more hyperparameters as needed
}

# Define the scoring metrics (negative mean absolute error and R²)
nae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
r2_scorer = 'r2'  # Scikit-learn's default r2_score is used here

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(gb_model, param_grid, scoring={'NAE': nae_scorer, 'R2': r2_scorer}, refit='NAE', cv=10)
grid_search.fit(X, y)

# Get the best hyperparameters
best_params_gb = grid_search.best_params_

# Get the best model
best_gb_model = grid_search.best_estimator_

# Use cross_val_score to get the overall performance of the best model (NAE)
overall_nae_performance_gb = cross_val_score(best_gb_model, X, y, cv=10, scoring=nae_scorer)

# Fit the best model to the entire dataset to get R²
best_gb_model.fit(X, y)
y_pred_gb = best_gb_model.predict(X)
r2_best_model_gb = r2_score(y, y_pred_gb)

# Print the results
print("Best Hyperparameters (Gradient Boosting):", best_params_gb)
print("Overall Negative Mean Absolute Error (Best Model):", np.mean(overall_nae_performance_gb))
print("R² for the Best Model (Gradient Boosting):", r2_best_model_gb)

Best Hyperparameters (Gradient Boosting): {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 150}
Overall Negative Mean Absolute Error (Best Model): -0.017882215168344553
R² for the Best Model (Gradient Boosting): 0.9999999999999812


In [38]:
y_test_pred = best_gb_model.predict(X_test)
print("test r2:",r2_score(y_test, y_test_pred))
print("test mae:",mean_absolute_error(y_test, y_test_pred))

test r2: 0.9999999999999811
test mae: 1.612851500464273e-08


In [39]:
#feature importance rf

# Get feature importances
feature_importances = best_gb_model.feature_importances_

#get feature names
feature_names = [f'{X.columns[i]}' for i in range(len(X.columns))]

# Create a DataFrame to display the feature importances along with the feature names
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the top 5 most important features
top_features = importance_df.head(5)
print("Top 5 Most Important Features:")
print(top_features)


Top 5 Most Important Features:
                 Feature  Importance
2                    GDP        0.59
3   XAUUSD Curncy - Gold        0.41
0                    CPI        0.00
1  Investment in Housing        0.00
4          Big China EtF        0.00


###Model 4: LSTM

In [40]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define the output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the LSTM layer
        out, _ = self.lstm(x)

        # Only take the output from the last time step
        out = out[:, -1, :]

        # Forward pass through the fully connected layer
        out = self.fc(out)

        return out

# Define the input size, hidden size, number of layers, and output size
input_size = 12  # Replace with your input size
hidden_size = 20  # Replace with your desired hidden size
num_layers = 3    # Number of LSTM layers
output_size = 1   # Replace with your output size

# Create the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Print the model architecture
print(model)

LSTMModel(
  (lstm): LSTM(12, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)


In [41]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Standardize the data (optional but often recommended for neural networks)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Convert the NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train_scaled).view(-1, 1, 12)  # Add batch and sequence dimensions
y_train_tensor = torch.Tensor(y_train_scaled)

X_test_tensor = torch.Tensor(X_test_scaled).view(-1, 1, 12)  # Add batch and sequence dimensions
y_test_tensor = torch.Tensor(y_test_scaled)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define the output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the LSTM layer
        out, _ = self.lstm(x)

        # Only take the output from the last time step
        out = out[:, -1, :]

        # Forward pass through the fully connected layer
        out = self.fc(out)

        return out

# Define the input size, hidden size, number of layers, and output size
input_size = 12  # Number of features in each time step
hidden_size = 64  # Replace with your desired hidden size
num_layers = 2    # Number of LSTM layers
output_size = 1   # LSTM output size (1 for regression)

# Create the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Print the model architecture
print(model)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model (you can adjust the number of epochs and other parameters)
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)

print(f'Test Loss: {test_loss.item():.4f}')

LSTMModel(
  (lstm): LSTM(12, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)
Epoch [10/100], Loss: 0.9677
Epoch [20/100], Loss: 0.8892
Epoch [30/100], Loss: 0.7482
Epoch [40/100], Loss: 0.5522
Epoch [50/100], Loss: 0.3961
Epoch [60/100], Loss: 0.3198
Epoch [70/100], Loss: 0.2361
Epoch [80/100], Loss: 0.1705
Epoch [90/100], Loss: 0.1249
Epoch [100/100], Loss: 0.0969
Test Loss: 0.3722


In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)

# Convert the predictions and true values back to the original scale
test_predictions = scaler_y.inverse_transform(np.array(test_outputs).reshape(-1, 1))
y_test_original = scaler_y.inverse_transform(np.array(y_test_tensor).reshape(-1, 1))

# Calculate negMAE, MSE, and R2
nmae = -mean_absolute_error(y_test_original, test_predictions)
mse = mean_squared_error(y_test_original, test_predictions)
r2 = r2_score(y_test_original, test_predictions)

print(f'Test Mean Squared Error (MSE): {mse:.4f}')
print(f'Test Negative Mean Absolute Error (negMAE): {nmae:.4f}')
print(f'Test R-squared (R2): {r2:.4f}')

Test Mean Squared Error (MSE): 0.0107
Test Negative Mean Absolute Error (negMAE): -0.0815
Test R-squared (R2): 0.4462
